In [0]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

In [1]:
import os
os.listdir('.')
os.chdir('drive/Colab Notebooks/EIP2/session04 densenet')
!ls 

1Densenet_CIFAR10.ipynb        weights01_28x28.0001-14.21.hdf5
2Densenet_CIFAR10.ipynb        weights02_28x28.0001-12.86.hdf5
3Densenet_CIFAR10.ipynb        weights03_28x28.0001-0.47.hdf5
3DenseNet_model_28x28.h5       weights03_28x28.0002-0.43.hdf5
4Densenet_CIFAR10.ipynb        weights04_28x28.0001-0.41.hdf5
4DenseNet_model_28x28.h5       weights04_28x28.0002-0.39.hdf5
5Densenet_CIFAR10.ipynb        weights04_28x28.0003-0.37.hdf5
5DenseNet_model_28x28.h5       weights05_28x28.0001-12.78.hdf5
6Densenet_CIFAR10.ipynb        weights05_28x28.0002-12.76.hdf5
pop2_3DenseNet_model_28x28.h5  weights05_28x28.0003-12.62.hdf5


In [2]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 14
num_filter = 20
compression = 0.75
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
img_height, img_width, channel

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [10]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)

output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
model.load_weights('5DenseNet_model_28x28.h5')

# determine Loss function and Optimizer
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
import os

checkpoint = ModelCheckpoint('weights06_28x28.{epoch:04d}-{val_loss:.2f}.hdf5',
                             monitor='val_loss', 
                             save_best_only=True)

callbacks_list = [checkpoint]

In [13]:
'''
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=True,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size), 
                    steps_per_epoch=len(x_train)/8, nb_epoch=epochs, verbose=1, callbacks = callbacks_list, 
                    validation_data=(x_test, y_test))

'''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,callbacks = callbacks_list, 
                    validation_data=(x_test, y_test))


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 286s 6ms/step - loss: 0.2123 - acc: 0.9257 - val_loss: 0.3642 - val_acc: 0.9013
Epoch 2/10
50000/50000 [==============================] - 265s 5ms/step - loss: 0.1650 - acc: 0.9428 - val_loss: 0.3419 - val_acc: 0.9060
Epoch 3/10
50000/50000 [==============================] - 264s 5ms/step - loss: 0.1518 - acc: 0.9462 - val_loss: 0.3497 - val_acc: 0.9043
Epoch 4/10
50000/50000 [==============================] - 265s 5ms/step - loss: 0.1435 - acc: 0.9497 - val_loss: 0.3498 - val_acc: 0.9065
Epoch 5/10
50000/50000 [==============================] - 265s 5ms/step - loss: 0.1377 - acc: 0.9512 - val_loss: 0.3473 - val_acc: 0.9078
Epoch 6/10
50000/50000 [==============================] - 264s 5ms/step - loss: 0.1315 - acc: 0.9540 - val_loss: 0.3456 - val_acc: 0.9088
Epoch 7/10
50000/50000 [==============================] - 264s 5ms/step - loss: 0.1272 - acc: 0.9557 - val_loss: 0.3421 - 

In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 23s 2ms/step
Test loss: 0.3444483554650098
Test accuracy: 0.9118


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("6DenseNet_model_28x28.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
!ls -l 

total 114056
-rw-r--r-- 1 root root   90274 Oct 26 06:45 1Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root   93263 Oct 26 07:04 2Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root   93283 Oct 26 07:27 3Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root 4165544 Oct 25 18:09 3DenseNet_model_28x28.h5
-rw-r--r-- 1 root root   93496 Oct 26 07:38 4Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root 4192544 Oct 25 22:43 4DenseNet_model_28x28.h5
-rw-r--r-- 1 root root   92996 Oct 26 07:47 5Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root 4194000 Oct 26 07:32 5DenseNet_model_28x28.h5
-rw-r--r-- 1 root root   89284 Oct 26 08:54 6Densenet_CIFAR10.ipynb
-rw-r--r-- 1 root root 4192544 Oct 26 08:54 6DenseNet_model_28x28.h5
-rw-r--r-- 1 root root 4154120 Oct 25 19:20 pop2_3DenseNet_model_28x28.h5
-rw-r--r-- 1 root root 7908264 Oct 25 16:16 weights01_28x28.0001-14.21.hdf5
-rw-r--r-- 1 root root 7908264 Oct 25 17:07 weights02_28x28.0001-12.86.hdf5
-rw-r--r-- 1 root root 7908264 Oct 25 18:05 weights03_28x28.0001-0.47.hdf5
-r